# implement line sweeps of params
- all params fixed expcet one, which is swept over range
- specifically looking at test accuracy as a function of one param


In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product

import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
## timestamp and dir for saving
tstamp = time.perf_counter_ns()
import os
os.mkdir('figures/scratch_folders/%i'%tstamp)

In [3]:
## default params
expargs = {
  'condition':'blocked',
  'n_train':160,
  'n_test':40
}
schargs = {
    'concentration':1.4,
    'stickiness_wi':5000,
    'stickiness_bt':5, # 100
    'sparsity':0.08,
    'pvar': 2,
    'lrate':0.8,
    'lratep':1,
    'decay_rate':1,
} 
args = {
    'sch':schargs,
    'exp':expargs
}
param_str = "-".join(["%s_%.3f"%(i,j) for i,j in schargs.items()])
param_str

'concentration_1.400-stickiness_wi_5000.000-stickiness_bt_5.000-sparsity_0.080-pvar_2.000-lrate_0.800-lratep_1.000-decay_rate_1.000'

In [78]:
gridD = {
  'concentration': [1.4,2.0,2.2],
  'stickiness_wi': [5000],
  'stickiness_bt': [5,10],
  'sparsity': [0.08],
  'pvar': [2],
  'lrate': [1],
  'lratep': [1],
  'decay_rate': [0.99],
}

gridD
## grid
pnames = [i for i in gridD.keys()]
pranges = [i for i in gridD.values()]
for idx,i in enumerate(product(*pranges)):
  d = dict(zip(pnames,i))
  
print('number of param conditions',idx)
print(d)

number of param conditions 5
{'concentration': 2.2, 'stickiness_wi': 5000, 'stickiness_bt': 10, 'sparsity': 0.08, 'pvar': 2, 'lrate': 1, 'lratep': 1, 'decay_rate': 0.99}


In [82]:
i,pnames
# dict(zip(pnames,i))

pvalstr = " ".join([str(j) for j in i])
out = "%s+%s"%(pnames,pvalstr)
out.split('+')[0]
pval

NameError: name 'pvals' is not defined

### main

In [5]:
def pltsave_macc(macc,schargs=None,labL=['B','I','E','M','L'],close=True):
  """ 
  save accuracy of individual parameter setting 
  used in paramsearch loops
  """
  plt.figure(figsize=(10,4))
  ax=plt.gca()
  for idx in range(len(macc)):
    ax.plot(macc[idx],label=labL[idx])
  ax.axhline(0.5,c='k')
  plt.legend()
  param_str = "-".join(["%s_%.3f"%(i,j) for i,j in schargs.items()])
  plt.savefig('figures/scratch_folders/%i/acc-%s.jpg'%(tstamp,param_str))
  if close:
    plt.close('all')
  return None


In [6]:
ns = 5
dfL = []
condL = ['blocked','interleaved','early','middle','late']
for idx,p_val in enumerate(p_vals):
  print(idx/len(p_vals))
  
  args['sch'][p_name] = p_val  
  exp_batch_data = run_batch_exp_curr(ns,args,condL)
  ## acc
  batch_acc = unpack_acc(exp_batch_data) # curr,seeds,trials
  mean_acc = batch_acc.mean(1)
  test_acc = mean_acc[:,-40:].mean(1) # curr  
  
  ## save traces of EML for each param setting
  pltsave_macc(mean_acc[2:],args['sch'],labL=['E','M','L'])
  
  ## record
  gsD = {
    **schargs,
    **dict(zip(condL,test_acc))
  }
  dfL.append(gsD)
  
gsdf = pd.DataFrame(dfL)

0.0
0.08333333333333333
0.16666666666666666
0.25
0.3333333333333333
0.4166666666666667
0.5
0.5833333333333334
0.6666666666666666
0.75
0.8333333333333334
0.9166666666666666


In [10]:
gsdf.to_csv('data/gsdf%i.csv'%tstamp)
gsdf ## test acc dataframe

,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,blocked,interleaved,early,middle,late
0,0.1,5000,5,0.08,2,0.8,1,1,0.52375,0.47875,0.51625,0.46000,0.51000
1,0.3,5000,5,0.08,2,0.8,1,1,0.47625,0.50375,0.49625,0.45875,0.51000
2,0.5,5000,5,0.08,2,0.8,1,1,0.49375,0.50375,0.55000,0.53875,0.49375
3,0.7,5000,5,0.08,2,0.8,1,1,0.49375,0.43875,0.48000,0.51875,0.50875
4,0.9,5000,5,0.08,2,0.8,1,1,0.50250,0.52000,0.51250,0.47750,0.52000
5,1.1,5000,5,0.08,2,0.8,1,1,0.52875,0.52250,0.55125,0.51000,0.52000
6,1.3,5000,5,0.08,2,0.8,1,1,0.50875,0.45625,0.47875,0.48250,0.46250
7,1.5,5000,5,0.08,2,0.8,1,1,0.51625,0.45125,0.52125,0.51500,0.53625
8,1.7,5000,5,0.08,2,0.8,1,1,0.54250,0.51750,0.50000,0.50375,0.52000
9,1.9,5000,5,0.08,2,0.8,1,1,0.50125,0.54750,0.47125,0.53250,0.46875


In [53]:
BI = gsdf.blocked > gsdf.interleaved
BM = gsdf.blocked > gsdf.middle
MI = gsdf.middle > gsdf.interleaved
gsdf[BI&BM&MI]

,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,blocked,interleaved,early,middle,late
6,1.3,5000,5,0.08,2,0.8,1,1,0.50875,0.45625,0.47875,0.4825,0.46250
7,1.5,5000,5,0.08,2,0.8,1,1,0.51625,0.45125,0.52125,0.5150,0.53625
